In [2]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# 21.10.2024
# 28.10.2024 Исправлена логика подсчёта ошибки и исправлена логика восстановления изображения из блоков

In [3]:
import numpy as np
from PIL import Image

In [4]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [5]:
def image_to_blocks(image, b_h, b_w, overlap = 0):
    i_h, i_w = image.shape[:2]

    step_h = int(b_h * (1 - overlap))
    step_w = int(b_w * (1 - overlap))

    blocks = []

    for i in range(0, i_h - b_h + 1, step_h):
        for j in range(0, i_w - b_w + 1, step_w):
            block = image[i:i+b_h, j:j+b_w]                                  
            blocks.append(block)    
    
    if i_h % b_h != 0:
        for j in range(0, i_w - b_w + 1, step_w):
            block = image[i_h-b_h:i_h, j:j+b_w]
            blocks.append(block)    
    
    if i_w % b_w != 0:
        for i in range(0, i_h - b_h + 1, step_h):
            block = image[i:i+b_h, i_w-b_w:i_w]
            blocks.append(block)    
    
    if i_h % b_h != 0 and i_w % b_w != 0:
        block = image[i_h-b_h:i_h, i_w-b_w:i_w]
        blocks.append(block)
    
    return np.asarray(blocks)


def blocks_to_image(image_blocks, image_shape, b_h, b_w, overlap = 0):
    i_h, i_w = image_shape[:2]
    c = image_shape[2] if len(image_shape) == 3 else 1

    restored_image = np.zeros((i_h, i_w, c), dtype=np.float64)
    count_matrix = np.zeros((i_h, i_w), dtype=np.float64)
    
    step_h = int(b_h * (1 - overlap))
    step_w = int(b_w * (1 - overlap))
    
    block_index = 0
    
    for i in range(0, i_h - b_h + 1, step_h):
        for j in range(0, i_w - b_w + 1, step_w):
            block = image_blocks[block_index]            
            restored_image[i:i+b_h, j:j+b_w] += block
            count_matrix[i:i+b_h, j:j+b_w] += 1
            block_index += 1    
    
    if i_h % b_h != 0:
        for j in range(0, i_w - b_w + 1, step_w):
            block = image_blocks[block_index]
            restored_image[i_h-b_h:i_h, j:j+b_w] += block
            count_matrix[i_h-b_h:i_h, j:j+b_w] += 1
            block_index += 1    
    
    if i_w % b_w != 0:
        for i in range(0, i_h - b_h + 1, step_h):
            block = image_blocks[block_index]
            restored_image[i:i+b_h, i_w-b_w:i_w] += block
            count_matrix[i:i+b_h, i_w-b_w:i_w] += 1
            block_index += 1    
    
    if i_h % b_h != 0 and i_w % b_w != 0:
        block = image_blocks[block_index]
        restored_image[i_h-b_h:i_h, i_w-b_w:i_w] += block
        count_matrix[i_h-b_h:i_h, i_w-b_w:i_w] += 1    
    
    count_matrix[count_matrix == 0] = 1    
    restored_image = restored_image / count_matrix[..., np.newaxis]
    restored_image[restored_image > 255] = 255    
    
    return restored_image.astype(np.uint8)

In [ ]:
def normalize_weights(weights):
    norms = np.linalg.norm(weights, axis=0)
    return weights / norms

# Функция активации
def linear_activation(x):
    return x

class LRNN:
    def __init__(self, input_dim, latent_dim, learning_rate=0.001):        
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.learning_rate = learning_rate        
        
        self.W_enc = normalize_weights(np.random.rand(self.input_dim, self.latent_dim))
        self.W_dec = normalize_weights(np.random.rand(self.latent_dim, self.input_dim))        

        self.epoch: int = 0
    
    def forward(self, x):
        z = linear_activation(x @ self.W_enc)
        x_reconstructed = linear_activation(z @ self.W_dec)
        return z, x_reconstructed
    
    def backward(self, x, x_reconstructed):
        error = x_reconstructed - x        
        
        dW_enc = (x.T @ error) @ self.W_dec.T
        dW_dec = (x @ self.W_enc).T @ error               
        
        self.W_dec -= self.learning_rate * dW_dec
        self.W_enc -= self.learning_rate * dW_enc        
        
        self.W_dec = normalize_weights(self.W_dec)
        self.W_enc = normalize_weights(self.W_enc)
    
    def squared_error(self, y_true, y_predicted) -> float:
        error = 0
        y_true, y_predicted = np.array(y_true)[0], np.array(y_predicted)[0]
        if len(y_true) != len(y_predicted):
            raise ValueError('True and predicted vectors must be same size!')
        for i in range(len(y_true)):
            error += (y_true[i] - y_predicted[i]) ** 2
        return error
    
    def train(self, data, epochs=1000, max_loss: float = 100, learn_by_loss: bool = False):
        for epoch in range(epochs):
            self.epoch += 1
            total_loss = 0
            for x in data:                
                x = np.matrix(x)
                _, x_reconstructed = self.forward(x)
                self.backward(x, x_reconstructed)
            for x in data:
                _, x_reconstructed = self.forward(x)
                total_loss += self.squared_error(x, x_reconstructed)
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss}')            
            if learn_by_loss and total_loss <= max_loss:
                break            

In [7]:
# Image compression/decompression pipeline
def compress_image(compression_weights, img_array, channels_amount: int,
                   block_height: int, block_width: int, overlap: float = 0):    
    normalized = (2.0 * img_array.astype(np.float32) / MAX_RGB_VALUE) - 1.0
    blocks = image_to_blocks(normalized, block_height, block_width, overlap)
    blocks = blocks.reshape((len(blocks), block_height * block_width, channels_amount))
    if channels_amount == 3:
        blocks = blocks.transpose(0, 2, 1)    
    blocks = np.einsum('ijk,kl->ijl', blocks, compression_weights)     
    return blocks
    

def decompress_image(decompression_weights, compressed_img, img_shape, channels_amount: int,
                     block_height: int, block_width: int, overlap: float = 0) -> Image.Image:
    compressed_img = np.einsum('ijk,kl->ijl', compressed_img, decompression_weights)
    compressed_img = MAX_RGB_VALUE * (compressed_img + 1.0) / 2.0
    if channels_amount == 3:
        compressed_img = compressed_img.transpose(0, 2, 1)
    compressed_img = compressed_img.reshape((len(compressed_img), block_height, block_width, channels_amount))    
    img_array = blocks_to_image(compressed_img, img_shape, block_height, block_width, overlap)    
    return Image.fromarray(img_array).convert('RGB' if channels_amount == 3 else 'L')

In [14]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 64 

img = Image.open('mountains.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, 10 * 10)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])


network = LRNN(100, 64, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1200)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, 10, 10, 0)
dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, 10, 10, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 1.5492138377310074
float64
Epoch 1/15000, Loss: 6004.547855722703
Epoch 2/15000, Loss: 5129.0333440928725
Epoch 3/15000, Loss: 4524.484768690819
Epoch 4/15000, Loss: 4071.706444716186
Epoch 5/15000, Loss: 3716.5241190386864
Epoch 6/15000, Loss: 3429.178326785791
Epoch 7/15000, Loss: 3191.3037297958604
Epoch 8/15000, Loss: 2990.6556826822707
Epoch 9/15000, Loss: 2818.6616104941045
Epoch 10/15000, Loss: 2669.159430933969
Epoch 11/15000, Loss: 2537.669776135855
Epoch 12/15000, Loss: 2420.9201772888905
Epoch 13/15000, Loss: 2316.5003427892734
Epoch 14/15000, Loss: 2222.6004609543506
Epoch 15/15000, Loss: 2137.815284624784
Epoch 16/15000, Loss: 2061.0062200469615
Epoch 17/15000, Loss: 1991.2129818394558
Epoch 18/15000, Loss: 1927.603056648343
Epoch 19/15000, Loss: 1869.4457484119087


KeyboardInterrupt: 

In [ ]:
# Test on another pic
img = Image.open('test5.jpg')
img_array = np.asarray(img)
shape = img_array.shape
compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, 10, 10, 0)
dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, 10, 10, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')